In [18]:
import requests
import json
import pandas as pd
from datetime import datetime 
from pprint import pprint 
import pytz

#Bruins Schedule Data
#api_url  = "https://statsapi.web.nhl.com/api/v1/schedule?teamId=6"

#general data about the current season
#api_url = "https://statsapi.web.nhl.com/api/v1/seasons/current"

#today's games
#api_url = "https://statsapi.web.nhl.com/api/v1/schedule"

#specific game data
#api_url = "https://statsapi.web.nhl.com/api/v1/game/2023010002/feed/live"

#API configurations
#api_url = "https://statsapi.web.nhl.com/api/v1/configurations"

#season game results
api_url = "https://statsapi.web.nhl.com/api/v1/schedule?season=20232024"

response = requests.get(api_url )
content = json.loads(response.content)
type(content)


dict

endpoint modifiers:
"https://statsapi.web.nhl.com/api/v1/schedule"
"https://statsapi.web.nhl.com/api/v1/schedule?date=2023-09-23"
schedule returns yesterday's games, adding ?date=2023-09-23 gives a certain date
https://statsapi.web.nhl.com/api/v1/schedule?season=20232024

In [19]:
# Send an HTTP GET request to the specified URL
response = requests.get(api_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # The response content can be accessed using response.text
    response_text = response.text
    #pprint(response_text)
else:
    print(f"Request failed with status code {response.status_code}")


In [20]:
# Parse the JSON string into a Python dictionary - what are the keys
json_data = json.loads(response_text)
json_data.keys()

dict_keys(['copyright', 'totalItems', 'totalEvents', 'totalGames', 'totalMatches', 'metaData', 'wait', 'dates'])

In [21]:
#get the dates and game numbers for the season
game_dates = json_data['dates']

#convert the game info dictionary into a dataframe
df_game_info= pd.DataFrame(game_dates)
df_game_info = df_game_info.convert_dtypes()
#df_game_info.head()

#Normalize (separates the data in the curly brackets out into separate columns)
game_data =pd.json_normalize(df_game_info['games'])
game_data= pd.DataFrame(game_data)
#game_data

#test
game_test1 =pd.json_normalize(game_data[0])
game_test1= pd.DataFrame(game_test1)
game_test1.head()



,gamePk,link,gameType,season,gameDate,status.abstractGameState,status.codedGameState,status.detailedState,status.statusCode,status.startTimeTBD,...,teams.home.leagueRecord.ot,teams.home.leagueRecord.type,teams.home.score,teams.home.team.id,teams.home.team.name,teams.home.team.link,venue.name,venue.link,content.link,venue.id
0,2023010001,/api/v1/game/2023010001/feed/live,PR,20232024,2023-09-23T04:05:00Z,Final,6,Final,6,False,...,0.0,league,5,53,Arizona Coyotes,/api/v1/teams/53,Rod Laver Arena,/api/v1/venues/null,/api/v1/game/2023010001/content,NaN
1,2023010004,/api/v1/game/2023010004/feed/live,PR,20232024,2023-09-24T04:05:00Z,Final,6,Final,6,False,...,1.0,league,3,26,Los Angeles Kings,/api/v1/teams/26,Rod Laver Arena,/api/v1/venues/null,/api/v1/game/2023010004/content,NaN
2,2023010016,/api/v1/game/2023010016/feed/live,PR,20232024,2023-09-25T18:00:00Z,Final,6,Final,6,False,...,0.0,league,5,13,Florida Panthers,/api/v1/teams/13,Amerant Bank Arena,/api/v1/venues/null,/api/v1/game/2023010016/content,NaN
3,2023010025,/api/v1/game/2023010025/feed/live,PR,20232024,2023-09-26T23:00:00Z,Final,6,Final,6,False,...,0.0,league,4,7,Buffalo Sabres,/api/v1/teams/7,KeyBank Center,/api/v1/venues/5039,/api/v1/game/2023010025/content,5039.0
4,2023010032,/api/v1/game/2023010032/feed/live,PR,20232024,2023-09-27T22:30:00Z,Final,6,Final,6,False,...,1.0,league,5,10,Toronto Maple Leafs,/api/v1/teams/10,Joe Thornton Community Centre,/api/v1/venues/null,/api/v1/game/2023010032/content,NaN


In [22]:
#create a loop to extract all of the data and put it in the dfs list

dfs = {}

# Loop through the iterations (30 times)
for i in range(0, 15):  # Loop from 1 to 30 (inclusive)
    # Make an API request and obtain the JSON response using the make_api_request function
    api_response = game_dates
    
    if api_response is not None:
        # Extract relevant data from the API response and normalize it
        game_info = pd.json_normalize(game_data[i])
        
        # Create a DataFrame for this iteration
        df_name = f'game_test{i}'  # Generate a unique variable name
        dfs[df_name] = pd.DataFrame(game_info)
    else:
        # Handle the case where the API request failed
        print(f"API request failed for index {i}")

# Now, you have 30 separate DataFrames stored in the 'dfs' dictionary
# You can access them using dfs['game_test1'], dfs['game_test2'], etc.
dfs['game_test1'].head()

,gamePk,link,gameType,season,gameDate,status.abstractGameState,status.codedGameState,status.detailedState,status.statusCode,status.startTimeTBD,...,teams.home.leagueRecord.ot,teams.home.leagueRecord.type,teams.home.score,teams.home.team.id,teams.home.team.name,teams.home.team.link,venue.id,venue.name,venue.link,content.link
0,2.023010e+09,/api/v1/game/2023010002/feed/live,PR,20232024,2023-09-23T19:00:00Z,Final,6,Final,6,False,...,0.0,league,3.0,19.0,St. Louis Blues,/api/v1/teams/19,5076.0,Enterprise Center,/api/v1/venues/5076,/api/v1/game/2023010002/content
1,2.023010e+09,/api/v1/game/2023010005/feed/live,PR,20232024,2023-09-24T17:00:00Z,Final,6,Final,6,False,...,1.0,league,3.0,5.0,Pittsburgh Penguins,/api/v1/teams/5,5034.0,PPG Paints Arena,/api/v1/venues/5034,/api/v1/game/2023010005/content
2,2.023010e+09,/api/v1/game/2023010017/feed/live,PR,20232024,2023-09-25T22:00:00Z,Final,6,Final,6,False,...,0.0,league,5.0,13.0,Florida Panthers,/api/v1/teams/13,NaN,Amerant Bank Arena,/api/v1/venues/null,/api/v1/game/2023010017/content
3,2.023010e+09,/api/v1/game/2023010026/feed/live,PR,20232024,2023-09-26T23:00:00Z,Final,6,Final,6,False,...,0.0,league,4.0,3.0,New York Rangers,/api/v1/teams/3,5054.0,Madison Square Garden,/api/v1/venues/5054,/api/v1/game/2023010026/content
4,2.023010e+09,/api/v1/game/2023010033/feed/live,PR,20232024,2023-09-27T23:00:00Z,Final,6,Final,6,False,...,0.0,league,4.0,12.0,Carolina Hurricanes,/api/v1/teams/12,5066.0,PNC Arena,/api/v1/venues/5066,/api/v1/game/2023010033/content


In [23]:
#Then I combine all of the dfs in the list by concatenation to create a single df
combined_df = pd.concat(dfs.values(), ignore_index=True)
combined_df = combined_df[['gamePk', 'gameType', 'gameDate', 'link']]
combined_df=combined_df.convert_dtypes()
combined_df['gameDate'] = pd.to_datetime(combined_df['gameDate'])
combined_df.head()

,gamePk,gameType,gameDate,link
0,2023010001,PR,2023-09-23 04:05:00+00:00,/api/v1/game/2023010001/feed/live
1,2023010004,PR,2023-09-24 04:05:00+00:00,/api/v1/game/2023010004/feed/live
2,2023010016,PR,2023-09-25 18:00:00+00:00,/api/v1/game/2023010016/feed/live
3,2023010025,PR,2023-09-26 23:00:00+00:00,/api/v1/game/2023010025/feed/live
4,2023010032,PR,2023-09-27 22:30:00+00:00,/api/v1/game/2023010032/feed/live


In [24]:
#add to the beginning of the link
prefix = 'https://statsapi.web.nhl.com'
combined_df['link'] = prefix + combined_df['link']
combined_df.head()

,gamePk,gameType,gameDate,link
0,2023010001,PR,2023-09-23 04:05:00+00:00,https://statsapi.web.nhl.com/api/v1/game/20230...
1,2023010004,PR,2023-09-24 04:05:00+00:00,https://statsapi.web.nhl.com/api/v1/game/20230...
2,2023010016,PR,2023-09-25 18:00:00+00:00,https://statsapi.web.nhl.com/api/v1/game/20230...
3,2023010025,PR,2023-09-26 23:00:00+00:00,https://statsapi.web.nhl.com/api/v1/game/20230...
4,2023010032,PR,2023-09-27 22:30:00+00:00,https://statsapi.web.nhl.com/api/v1/game/20230...


##### How to pull out single game data

In [25]:
#testing out on a specific game
game_url = "https://statsapi.web.nhl.com/api/v1/game/2023010002/feed/live"

# Send an HTTP GET request to the specified URL
response2 = requests.get(game_url)

# Check if the request was successful (status code 200)
if response2.status_code == 200:
    # The response content can be accessed using response.text
    response_text2 = response2.text
    #pprint(response_text)
else:
    print(f"Request failed with status code {response2.status_code}")

    #getting in-game data
json_data2 = json.loads(response_text2)
event_data = json_data2['liveData']['plays']['allPlays']

#convert the dictionary into a dataframe
event_data = pd.DataFrame(event_data)

#Normalize (separates the data in the curly brackets out into separate columns)

#coordinates column data
coordinates=pd.json_normalize(event_data['coordinates'])
coordinates= pd.DataFrame(coordinates)

#results column data
game_events = pd.json_normalize(event_data['result'])
game_events=pd.DataFrame(game_events)
game_events = game_events[['event','eventCode', 'eventTypeId', 'description']]
game_events['eventId'] = game_events['eventCode'].str[3:].astype(int)

#about column data
about_game = pd.json_normalize(event_data['about'])
about_game=pd.DataFrame(about_game)
about_game = about_game[['eventIdx', 'eventId', 'periodTime', 'periodTimeRemaining']]

#players column data - bringing it in as a dictionary first to connect the data to the event id before parsing
event_players = pd.DataFrame(event_data['players'])
event_players['eventIdx'] = event_players.index

#merge all three dfs together to create a new single df, merging left onto game events
all_game_data = game_events.merge(about_game, on="eventId", how = "left")

#adding in coordinates data on the index
all_game_data = all_game_data.merge(coordinates, left_index=True, right_index=True)

#merging in the player data
all_game_data = all_game_data.merge(event_players, left_index=True, right_index=True)

#extracting the player ID data to connect it to the event id
def extract_player_id(player_data):
    if isinstance(player_data, list) and len(player_data) > 0:
        player = player_data[0]
        if 'player' in player and 'id' in player['player']:
            return player['player']['id']
    return None

# Apply the function to create a new 'player_id' column
all_game_data['player_id'] = all_game_data['players'].apply(extract_player_id)

# Convert the 'player_id' column to the appropriate data type (e.g., int)
all_game_data['player_id'] = all_game_data['player_id'].astype(float).astype(pd.Int64Dtype(), errors='ignore')

all_game_data.head()

,event,eventCode,eventTypeId,description,eventId,eventIdx_x,periodTime,periodTimeRemaining,x,y,players,eventIdx_y,player_id
0,Game Scheduled,STL1,GAME_SCHEDULED,Game Scheduled,1,0,00:00,20:00,NaN,NaN,NaN,0,<NA>
1,Period Ready,STL5,PERIOD_READY,Period Ready,5,1,00:00,20:00,NaN,NaN,NaN,1,<NA>
2,Period Start,STL51,PERIOD_START,Period Start,51,2,00:00,20:00,NaN,NaN,NaN,2,<NA>
3,Faceoff,STL53,FACEOFF,Nathan Smith faceoff won against Robert Thomas,53,3,00:00,20:00,0.0,0.0,"[{'player': {'id': 8480979, 'fullName': 'Natha...",3,8480979
4,Hit,STL8,HIT,Milos Kelemen hit Oskar Sundqvist,8,4,01:23,18:37,87.0,37.0,"[{'player': {'id': 8483597, 'fullName': 'Milos...",4,8483597


In [26]:
#Extracting the game id from the url
# Split the URL by "game/" and take the last part
#parts = game_url.split("game/")
#if len(parts) > 1:
#    game_id_with_extra = parts[-1]  # This includes "/feed/live"
#    # Remove the "/feed/live" portion
#    game_id = game_id_with_extra.split("/feed/live")[0]

will look like this: 2023020001
- The first 4 digits identify the season of the game (ie. 2017 for the 2017-2018 season). Always refer to a season with the starting year. A game played in March 2018 would still have a game ID that starts with 2017
- The next 2 digits give the type of game, where 01 = preseason, 02 = regular season, 03 = playoffs, 04 = all-star
- The final 4 digits identify the specific game number. For regular season and preseason games, this ranges from 0001 to the number of games played. (1353 for seasons with 32 teams (2022 - Present), 1271 for seasons with 31 teams (2017 - 2020) and 1230 for seasons with 30 teams). For playoff games, the 2nd digit of the specific number gives the round of the playoffs, the 3rd digit specifies the matchup, and the 4th digit specifies the game (out of 7).

To schedule and run a Python script daily on its own, you can use various methods depending on your operating system and requirements. Here are two common approaches:

1. **Using Cron Jobs (Linux/macOS)**:

    - On Linux and macOS, you can schedule your Python script using cron jobs.
    
    - Open your terminal and type `crontab -e` to edit your user's cron jobs.
    
    - Add a new line to schedule your Python script. For example, to run your script every day at 3:00 AM, you can add:
    
      ```shell
      0 3 * * * /usr/bin/python3 /path/to/your/script.py
      ```

    - Save and exit the editor. The script will run automatically every day at the specified time.

2. **Using Task Scheduler (Windows)**:

    - On Windows, you can use Task Scheduler to schedule your Python script.
    
    - Open Task Scheduler from the Start menu or search.
    
    - In the right pane, click "Create Basic Task" to open the wizard.
    
    - Follow the wizard to specify a name, trigger (daily), start time, and frequency.
    
    - In the "Action" step, choose "Start a program" and browse to select your Python script (e.g., `python.exe` and your script path).
    
    - Complete the wizard, and your script will run daily as scheduled.

3. **Using Third-Party Services**:

    - You can use cloud-based services like AWS Lambda, Google Cloud Functions, or Azure Functions to schedule and run your Python script. These services allow you to set up cron-like schedules for your code to execute in a serverless environment.

4. **Using Python Libraries**:

    - You can use Python libraries like `schedule` or `APScheduler` to schedule and run your script at specific times. This approach is suitable if you want to keep the scheduling logic within your Python script.

Choose the method that best suits your operating system and requirements. Make sure to provide the correct path to your Python script, and ensure that the necessary dependencies are installed for your script to run successfully.

##### Creating a roster df - this is for last season bc this season does not have a roster set yet. Must change in october

In [27]:
stats_url = "https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster&season=20222023"

response = requests.get(stats_url )
content = json.loads(response.content)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # The response content can be accessed using response.text
    response_text = response.text
    #pprint(response_text)
else:
    print(f"Request failed with status code {response.status_code}")

json_data = json.loads(response_text)
team_data = json_data['teams']

#this is extracting the data from a single team, the number in the [] indicates which team in the database to extract roster info for. Bruins are team 6
# Assuming you have the JSON response as 'response_text'
#json_data = json.loads(response_text)
# Extract 'roster' data from the 'teams' key
#roster_data = json_data['teams'][0]['roster']['roster']
# Create a DataFrame directly from the 'roster' data
#person_df = pd.json_normalize(roster_data)
#person_df
# Assuming you have a list of teams in your data, you can iterate through them
all_rosters = []

for team in json_data['teams']:
    team_id = team['id']  # Get the team ID
    team_name = team['name']  # Get the team name

    # Extract the roster for the current team
    roster_data = team['roster']['roster']

    # Create a DataFrame for the roster data
    roster_df = pd.json_normalize(roster_data)

    # Add a 'team_id' and 'team_name' column to identify the team
    roster_df['team_id'] = team_id
    roster_df['team_name'] = team_name

    # Append the roster data for the current team to the list
    all_rosters.append(roster_df)

# Concatenate all the roster DataFrames into one
final_roster_data = pd.concat(all_rosters, ignore_index=True)
final_roster_data.head()

,jerseyNumber,person.id,person.fullName,person.link,position.code,position.name,position.type,position.abbreviation,team_id,team_name
0,2,8474090,Brendan Smith,/api/v1/people/8474090,D,Defenseman,Defenseman,D,1,New Jersey Devils
1,90,8475193,Tomas Tatar,/api/v1/people/8475193,L,Left Wing,Forward,LW,1,New Jersey Devils
2,56,8475287,Erik Haula,/api/v1/people/8475287,L,Left Wing,Forward,LW,1,New Jersey Devils
3,18,8476292,Ondrej Palat,/api/v1/people/8476292,L,Left Wing,Forward,LW,1,New Jersey Devils
4,7,8476462,Dougie Hamilton,/api/v1/people/8476462,D,Defenseman,Defenseman,D,1,New Jersey Devils


#### developing the final df

In [34]:
# Create a timezone object for UTC-4
utc_minus_4 = pytz.timezone('America/New_York')  # Adjust to the correct timezone if necessary

# Get the current date in UTC-4 timezone
today_date_utc_minus_4 = datetime.now(utc_minus_4).date()

# Assuming your 'gameDate' column is already in UTC-4, you can directly compare it
filtered_games = games[games['gameDate'].dt.date < today_date_utc_minus_4]

filtered_games = filtered_games.rename(columns={'gamePk': 'game_id'})

# games_to_date now contains games that have happened prior to today in UTC-4 timezone
games_to_date = filtered_games

In [35]:
# Assuming your DataFrame is named 'games_to_date'
games_to_date = games_to_date.sort_values(by='game_id')
games_to_date = games_to_date.reset_index(drop=True)
games_to_date

,game_id,gameType,gameDate,link
0,2023010001,PR,2023-09-23 04:05:00+00:00,https://statsapi.web.nhl.com/api/v1/game/20230...
1,2023010002,PR,2023-09-23 19:00:00+00:00,https://statsapi.web.nhl.com/api/v1/game/20230...
2,2023010003,PR,2023-09-24 00:00:00+00:00,https://statsapi.web.nhl.com/api/v1/game/20230...
3,2023010004,PR,2023-09-24 04:05:00+00:00,https://statsapi.web.nhl.com/api/v1/game/20230...
4,2023010005,PR,2023-09-24 17:00:00+00:00,https://statsapi.web.nhl.com/api/v1/game/20230...
...,...,...,...,...
63,2023010064,PR,2023-10-01 02:00:00+00:00,https://statsapi.web.nhl.com/api/v1/game/20230...
64,2023010065,PR,2023-10-01 20:00:00+00:00,https://statsapi.web.nhl.com/api/v1/game/20230...
65,2023010066,PR,2023-10-01 22:00:00+00:00,https://statsapi.web.nhl.com/api/v1/game/20230...
66,2023010067,PR,2023-10-01 23:00:00+00:00,https://statsapi.web.nhl.com/api/v1/game/20230...


In [36]:

# Create an empty dictionary to store the results
game_data_dict = {}

    
    # Define your extraction script as a function
def extract_game_data(game_link):
    try:
        # Send an HTTP GET request to the game URL
        response = requests.get(game_link)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            response_text = response.text
            json_game_data = json.loads(response_text)
            
            event_data = json_game_data['liveData']['plays']['allPlays']
            event_data = pd.DataFrame(event_data)
            coordinates = pd.json_normalize(event_data['coordinates'])
            coordinates = pd.DataFrame(coordinates)
            game_events = pd.json_normalize(event_data['result'])
            game_events = pd.DataFrame(game_events)
            game_events = game_events[['event', 'eventCode', 'eventTypeId', 'description']]
            game_events['eventId'] = game_events['eventCode'].str[3:].astype(int)
            about_game = pd.json_normalize(event_data['about'])
            about_game = pd.DataFrame(about_game)
            about_game = about_game[['eventIdx', 'eventId', 'periodTime', 'periodTimeRemaining']]
            all_game_data = game_events.merge(about_game, on="eventId", how="left")
            all_game_data = all_game_data.merge(coordinates, left_index=True, right_index=True)
            event_players = pd.DataFrame(event_data['players'])
            event_players['eventIdx'] = event_players.index
            all_game_data = all_game_data.merge(event_players, left_index=True, right_index=True)
     
            # Define a function to extract player IDs safely
            def extract_player_id(player_data):
                if isinstance(player_data, list) and len(player_data) > 0:
                    player = player_data[0]
                    if 'player' in player and 'id' in player['player']:
                        return player['player']['id']
                return None

            # Apply the function to create a new 'player_id' column
            all_game_data['player_id'] = all_game_data['players'].apply(extract_player_id)

            # Convert the 'player_id' column to the appropriate data type (e.g., int)
            all_game_data['player_id'] = all_game_data['player_id'].astype(float).astype(pd.Int64Dtype(), errors='ignore')
            goals = all_game_data.query('eventTypeId == "GOAL"')
            
            return goals  # Return the entire processed DataFrame
        else:
            print(f"Request failed for {game_link} with status code {response.status_code}")
            return None  # Return None to indicate failure
    except Exception as e:
        print(f"An error occurred: {e}")
        return None  # Return None to indicate failure
    
    
# Loop through the rows of the games_to_date df
for index, row in games_to_date.iterrows():
    # Extract the API link from the current row
    game_link = row['link']
    
 # Run your game-specific data script and get the entire processed DataFrame
    game_specific_data = extract_game_data(game_link)
    
    # Store the result in the dictionary with the game ID as the key
    game_data_dict[row['game_id']] = game_specific_data



In [37]:
game_data_dict.keys()

dict_keys([2023010001, 2023010002, 2023010003, 2023010004, 2023010005, 2023010006, 2023010007, 2023010008, 2023010009, 2023010010, 2023010011, 2023010012, 2023010013, 2023010014, 2023010015, 2023010016, 2023010017, 2023010018, 2023010019, 2023010020, 2023010021, 2023010022, 2023010023, 2023010024, 2023010025, 2023010026, 2023010027, 2023010028, 2023010029, 2023010030, 2023010031, 2023010032, 2023010033, 2023010034, 2023010035, 2023010036, 2023010037, 2023010038, 2023010039, 2023010040, 2023010041, 2023010042, 2023010043, 2023010044, 2023010045, 2023010046, 2023010047, 2023010048, 2023010049, 2023010050, 2023010051, 2023010052, 2023010053, 2023010054, 2023010055, 2023010056, 2023010057, 2023010058, 2023010059, 2023010060, 2023010061, 2023010062, 2023010063, 2023010064, 2023010065, 2023010066, 2023010067, 2023010068])

In [38]:
# Loop through the rows of the games_to_date df
for index, row in games_to_date.iterrows():
    # Extract the API link from the current row
    game_link = row['link']
    
    # Run your game-specific data script and get the entire processed DataFrame
    game_specific_data = extract_game_data(game_link)
    
    # Add a 'game_id' column to the game_specific_data DataFrame
    game_specific_data['game_id'] = row['game_id']
    
    # Store the result in the dictionary with the game ID as the key
    game_data_dict[row['game_id']] = game_specific_data
game_data_dict

{2023010001:    event eventCode eventTypeId  \
 5   Goal   ARI9006        GOAL   
 7   Goal   ARI9010        GOAL   
 8   Goal   ARI9013        GOAL   
 9   Goal   ARI9016        GOAL   
 10  Goal   ARI9019        GOAL   
 12  Goal   ARI9023        GOAL   
 13  Goal   ARI9026        GOAL   
 17  Goal   ARI9032        GOAL   
 
                                           description  eventId  eventIdx_x  \
 5   Clayton Keller (1) , assists: Nick Schmaltz (1...     9006           5   
 7   Matias Maccelli (1) , assists: Juuso Valimaki ...     9010           7   
 8       Adrian Kempe (1) , assists: Jordan Spence (1)     9013           8   
 9      Lawson Crouse (1) , assists: Nick Bjugstad (1)     9016           9   
 10  Logan Cooley (1) , assists: J.J. Moser (1), Dy...     9019          10   
 12    Viktor Arvidsson (1) , assists: Kevin Fiala (1)     9023          12   
 13   Trevor Moore (1) , assists: Viktor Arvidsson (1)     9026          13   
 17  Barrett Hayton (1) , assists: Clay

In [46]:
# Create an empty dictionary to store the results
game_data_dict = {}

# Loop through the rows of the games_to_date df
for index, row in games_to_date.iterrows():
    # Extract the API link from the current row
    game_link = row['link']
    
    # Run your game-specific data script and get the entire processed DataFrame
    game_specific_data = extract_game_data(game_link)
    
    # Set the 'game_id' for the game_specific_data using the current row's 'game_id'
    if game_specific_data is not None:
        game_specific_data['game_id'] = row['game_id']
    
    # Store the result in the dictionary with the game ID as the key
    game_data_dict[row['game_id']] = game_specific_data

# Combine all the game-specific DataFrames in the dictionary into one DataFrame
final_game_data = pd.concat(game_data_dict.values(), ignore_index=True)
final_game_data.to_csv( 'final_game_data.csv', index=False) 

#### Creating a df to track players' goals and coordinates

In [40]:
final_roster_data.head()

,jerseyNumber,person.id,person.fullName,person.link,position.code,position.name,position.type,position.abbreviation,team_id,team_name
0,2,8474090,Brendan Smith,/api/v1/people/8474090,D,Defenseman,Defenseman,D,1,New Jersey Devils
1,90,8475193,Tomas Tatar,/api/v1/people/8475193,L,Left Wing,Forward,LW,1,New Jersey Devils
2,56,8475287,Erik Haula,/api/v1/people/8475287,L,Left Wing,Forward,LW,1,New Jersey Devils
3,18,8476292,Ondrej Palat,/api/v1/people/8476292,L,Left Wing,Forward,LW,1,New Jersey Devils
4,7,8476462,Dougie Hamilton,/api/v1/people/8476462,D,Defenseman,Defenseman,D,1,New Jersey Devils


In [45]:
goal_tracker = final_roster_data[['jerseyNumber', 'person.id', 'person.fullName', 'position.name', 'team_name']]
# Check if 'person.id' exists in goal_tracker DataFrame
if 'person.id' in goal_tracker.columns:
    # Merge the coordinates data from final_game_data to goal_tracker based on player IDs
    goal_tracker = goal_tracker.merge(final_game_data[['player_id', 'x', 'y']], left_on='person.id', right_on='player_id', how='left')
else:
    print("Column 'person.id' does not exist in goal_tracker DataFrame.")

# Use the to_csv method to export the DataFrame to a CSV file
goal_tracker.to_csv( 'goal_tracker.csv', index=False) 